<a href="https://colab.research.google.com/github/amirmsk/IELTS_Listening_assesment/blob/main/IELTS_Listening.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-telegram-bot --upgrade
!pip install edge-tts
!pip install -U langchain-groq
!pip install pydub
!pip install pydantic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 702.3/702.3 kB 9.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for srt: filename=srt-3.5.3-py3-none-any.whl size=22427 sha256=2c326847f3cfe2b05872acab59f8fec24cff1bd39a1b3938ff18647bcc56e78a
  Stored in directory: /root/.cache/pip/wheels/1f/43/f1/23ee9119497fcb57d9f7046fbf34c6d9027c46a1fa7824cf08
Successfully built srt
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.6/129.6 kB 3.9 MB/s eta 0:00:00


In [ ]:
# Standard libraries
import os
import json
import base64
import asyncio
import textwrap
import logging
import re
import random
from pathlib import Path
from collections import defaultdict
from typing import List, Literal

# Third-party libraries
import nest_asyncio
import edge_tts
from pydantic import BaseModel, Field, ValidationError

# Telegram Bot API
from telegram import Update, InlineKeyboardButton, InlineKeyboardMarkup
from telegram.ext import ApplicationBuilder, CommandHandler, CallbackQueryHandler, ContextTypes

# LangChain
from langchain.chat_models import init_chat_model
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import Runnable
from langchain.output_parsers import JsonOutputKeyToolsParser

# ─── Credentials ──────────────────────────────────────────────────
BOT_TOKEN = "8184053977:AAH-gGeCr0Mg5eMEyvWFWQ2ldvafBDpEhI8"
os.environ["GROQ_API_KEY"] = "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"

model = init_chat_model(
    model="llama-3.3-70b-versatile",
    model_provider="groq",
    temperature=1.5,
)

# ─── Global State ─────────────────────────────────────────────────
user_state: dict[int, dict] = {}

# You can extend these later if you like
MALE_VOICES   = ["en-US-GuyNeural", "en-US-ChristopherNeural"]
FEMALE_VOICES = ["en-US-AriaNeural", "en-US-JennyNeural"]
DEFAULT_VOICE = "en-US-AriaNeural"

# ─── Helper Functions ─────────────────────────────────────────────
async def ask_llm(prompt: str) -> str:
    def sync_call():
        resp = model.invoke(prompt)
        output = getattr(resp, "content", str(resp))
        print("LLM raw output:\n", output)  # Debug print
        return output
    return await asyncio.to_thread(sync_call)

def parse_mcqs(raw_mcq: str) -> list[dict]:
    mcqs = []
    blocks = [b.strip() for b in raw_mcq.strip().split("\n\n") if b.strip()]
    i = 0
    while i < len(blocks):
        lines = [ln.strip() for ln in blocks[i].split("\n") if ln.strip()]
        # Merge answer line if it’s in next block
        if len(lines) == 5 and i + 1 < len(blocks) and re.match(r"(?i)^Answer:\s*[a-d]$", blocks[i + 1].strip()):
            lines.append(blocks[i + 1].strip())
            i += 1
        if len(lines) < 6:
            i += 1
            continue
        question = lines[0]
        options = lines[1:5]
        answer_letter = lines[5].split(":")[-1].strip().lower()
        mcqs.append({"question": question, "options": options, "answer": answer_letter})
        i += 1
    return mcqs

def is_base64(s):
    try:
        # Adjust string padding to be multiple of 4
        s += '=' * (-len(s) % 4)
        base64.b64decode(s)
        return True
    except Exception:
        return False


def create_speaker_voice_map(dialogue):
    voice_map = {}
    used_voices = set()

    for turn in dialogue:  # <-- just iterate over dialogue directly since it's a list
        speaker = getattr(turn, "name", "Unknown")
        gender = getattr(turn, "gender", None)

        if speaker not in voice_map:
            if gender == "male":
                voices = [v for v in MALE_VOICES if v not in used_voices]
            elif gender == "female":
                voices = [v for v in FEMALE_VOICES if v not in used_voices]
            else:
                voices = [v for v in MALE_VOICES + FEMALE_VOICES if v not in used_voices]

            voice = voices[0] if voices else "en-US-JennyNeural"
            voice_map[speaker] = voice
            used_voices.add(voice)

    voice_map["default"] = "en-US-JennyNeural"
    return voice_map

async def send_mcqs(chat_id: int, mcqs: list[dict], context: ContextTypes.DEFAULT_TYPE):
    for mcq in mcqs:
        options_text = "\n".join(mcq["options"])
        await context.bot.send_message(chat_id, f"{mcq['question']}\n\n{options_text}")
        buttons = [InlineKeyboardButton(opt[0], callback_data=opt[0].lower()) for opt in mcq["options"]]
        await context.bot.send_message(chat_id, "Select the correct option:", reply_markup=InlineKeyboardMarkup([buttons]))


async def tts_to_file(dialogue: list, voice_map: dict, output_file: str) -> str:
    combined_audio = b""

    for turn in dialogue:
        text = turn.text.strip()
        speaker = turn.name
        voice = voice_map.get(speaker, voice_map.get("default", "en-US-JennyNeural"))

        try:
            communicate = edge_tts.Communicate(text=text, voice=voice)
            async for chunk in communicate.stream():
                if chunk["type"] == "audio":
                    combined_audio += chunk["data"]
        except Exception as e:
            print(f"⚠️  TTS failed for speaker '{speaker}': {e}")

    Path(output_file).write_bytes(combined_audio)
    return output_file


# Define your Dialogue and Conversation models
class Dialogue(BaseModel):
    name: str = Field(description="The name of the speaker")
    gender: str = Field(description="The gender of the speaker (male or female)")
    text: str = Field(description="The actual spoken line")

class Conversation(BaseModel):
    dialogues: List[Dialogue]

# Assume you have a model wrapper created somewhere like this:
structured_ask_passage = model.with_structured_output(Conversation)

async def generate_passage_and_mcqs(chat_id: int, context: ContextTypes.DEFAULT_TYPE, stage: str, section: str):
    prev = user_state[chat_id].get('previous_passage', '')

    # Build the prompt based on section
    if section == 'section1':
        passage_prompt = textwrap.dedent(f"""
            You are an IELTS Listening passage writer.
            Continue the dialogue between two speakers lasting about two minutes (≈300 words), it should be about everyday conversations.
            It should be related in theme and context to this previous passage: {prev}
            Return only the dialogue lines in the structure:
        """)
    elif section == 'section2':
        passage_prompt = textwrap.dedent(f"""
            You are an IELTS Listening passage writer.
            Create a monologue lasting about two minutes (≈300 words) on a general or semi-formal topic.
            It should be related in theme and context to this previous monologue: {prev}
            Return only the monologue lines in the structure:
        """)
    elif section == 'section3':
        passage_prompt = textwrap.dedent(f"""
            You are an IELTS Listening passage writer.
            Create a conversation among 2 to 4 speakers, lasting about two minutes (≈300 words), about an academic or training-related topic.
            It should be related in theme and context to this previous conversation: {prev}
        """)
    elif section == 'section4':
        passage_prompt = textwrap.dedent(f"""
            You are an IELTS Listening passage writer.
            Create a lecture or academic talk delivered by a single speaker, lasting about two minutes (≈300 words), on a university-level topic.
            It should relate to this previous lecture: {prev}
        """)
    else:
        await context.bot.send_message(chat_id, "❌ Invalid section. Cannot generate passage.")
        return

    logger.debug(f"Passage prompt:\n{passage_prompt}")

    # === Generate structured passage using with_structured_output and ainvoke() ===
    try:
        structured_convo: Conversation = await structured_ask_passage.ainvoke(passage_prompt)
        dialogue = structured_convo.dialogues
        user_state[chat_id]['previous_passage'] = json.dumps(structured_convo.dict(), indent=2, ensure_ascii=False)
        logger.info("✅ Generated and validated structured passage.")
    except ValidationError as e:
        logger.error(f"Failed to get or validate passage for chat_id={chat_id}: {e}")
        await context.bot.send_message(chat_id, "❌ Failed to generate a valid structured passage. Please try again.")
        return
    except Exception as e:
        logger.error(f"Unexpected error during passage generation: {e}")
        await context.bot.send_message(chat_id, "❌ Unexpected error during passage generation.")
        return

    # === Voice synthesis ===
    try:
        voice_map = create_speaker_voice_map(dialogue)
        audio_file = f"passage_{chat_id}_{stage}.mp3"
        audio_path = await tts_to_file(dialogue, voice_map, audio_file)
        with open(audio_path, 'rb') as af:
            await context.bot.send_audio(chat_id, audio=af, title=f"Listening Passage 🎧 {stage.capitalize()}")
    except Exception as e:
        logger.error(f"Failed to generate or send TTS audio: {e}")
        await context.bot.send_message(chat_id, "❌ Failed to generate or send audio. Please try again.")
        return

    # === Generate MCQs ===
    mcq_prompt = textwrap.dedent(f"""
        You are an IELTS listening MCQ writer.
        Based on the passage below, create four multiple-choice questions with four options each (a-d).
        Mark the correct option letter on a separate line starting with "Answer:".
        Passage:
        {user_state[chat_id]['previous_passage']}
        Format:
        Q1: ...
        a) ...
        b) ...
        c) ...
        d) ...
        Answer:
        Q2: ...
        a) ...
        b) ...
        c) ...
        d) ...
        Answer:
        Q3: ...
        a) ...
        b) ...
        c) ...
        d) ...
        Answer:
        Q4: ...
        a) ...
        b) ...
        c) ...
        d) ...
        Answer:
    """)

    logger.debug(f"MCQ prompt:\n{mcq_prompt}")

    try:
        mcq_raw = await ask_llm(mcq_prompt)
        mcqs = parse_mcqs(mcq_raw)
        user_state[chat_id]['mcq_correct'] = [m['answer'] for m in mcqs]
        user_state[chat_id]['mcq_user'] = []

        await send_mcqs(chat_id, mcqs, context)
        logger.info(f"✅ MCQs sent for chat_id={chat_id}")
    except Exception as e:
        logger.error(f"Failed to generate or send MCQs for chat_id={chat_id}: {e}")
        await context.bot.send_message(chat_id, "❌ Failed to generate MCQs. Please try again.")
        return

    logger.info(f"🎯 Finished generating passage and MCQs for chat_id={chat_id}, stage={stage}, section={section}")

# ─── Command Handlers ─────────────────────────────────────────────
# Assuming you have this model defined somewhere

# Define dialogue and conversation schemas
class Dialogue(BaseModel):
    name: str = Field(description="The name of the speaker")
    gender: str = Field(description="The gender of the speaker (male or felmale)")
    text: str = Field(description="The actual spoken line")

class Conversation(BaseModel):
    dialogues: List[Dialogue]

# Setup your LLM to return structured Conversation schema
structured_ask_dialogue = model.with_structured_output(Conversation)

async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    chat_id = update.effective_chat.id
    user_state[chat_id] = {
        'step': 'mcq',
        'section': 'section1',
        'stage': 'part1',
        'previous_passage': '',
        'mcq_correct': [],
        'mcq_user': [],
        'tscore': 0
    }

    await context.bot.send_message(chat_id, "Welcome! Starting Stage 1 Part 1 listening practice...")

    conversation_examples = [
        "buying groceries at a supermarket",
        "booking a hotel room over the phone",
        "calling a taxi to go to the airport",
        "inquiring about an apartment for rent",
        "ordering food in a restaurant",
        # ... (rest can remain)
    ]
    example = random.choice(conversation_examples)
    print("🎯 Selected topic:", example)

    # Generate structured dialogue from LLM
    try:
        structured_convo: Conversation = await structured_ask_dialogue.ainvoke(textwrap.dedent(f"""
            You are an IELTS Listening passage writer.
            Create conversation between two speakers lasting about two minutes (~300 words).
            Topic: "{example}"
        """))
        dialogue = structured_convo.dialogues
    except ValidationError as e:
        print("❌ Dialogue validation error:", e)
        await context.bot.send_message(chat_id, "❌ Failed to generate a valid structured dialogue. Please try again.")
        return

    # Dump structured dialogue as JSON
    dialogue_json = json.dumps(structured_convo.dict(), indent=2, ensure_ascii=False)
    print("✅ Structured JSON dialogue:\n", dialogue_json)

    user_state[chat_id]['previous_passage'] = dialogue_json

    # 🔧 Assign speaker voices
    voice_map = create_speaker_voice_map(dialogue)
    audio_file = f"passage_{chat_id}_part1.mp3"
    audio_path = await tts_to_file(dialogue, voice_map, audio_file)

    with open(audio_path, 'rb') as af:
        await context.bot.send_audio(chat_id, audio=af, title="Listening Passage 🎧 Part 1")

    # ✍️ Generate MCQs from structured JSON dialogue
    mcq_prompt = textwrap.dedent(f"""
        You are an IELTS listening MCQ writer.
        Based on the following structured dialogue (in JSON format), create four multiple-choice questions (Q1–Q4).
        Each question should have four options (a–d). Include correct answers after each question using "Answer: x".

        Dialogue:
        {dialogue_json}

        Format:
        Q1: ...
        a) ...
        b) ...
        c) ...
        d) ...
        Answer: b
    """)
    mcq_response = await ask_llm(mcq_prompt)
    mcqs = parse_mcqs(mcq_response)

    user_state[chat_id]['mcq_correct'] = [m['answer'] for m in mcqs]
    user_state[chat_id]['mcq_user'] = []

    await send_mcqs(chat_id, mcqs, context)

async def handle_button(update: Update, context: ContextTypes.DEFAULT_TYPE):
    query = update.callback_query
    await query.answer()
    chat_id = query.message.chat.id
    state = user_state.get(chat_id)

    if not state or state.get('step') != 'mcq':
        return

    # Append user's answer
    state['mcq_user'].append(query.data)
    await query.edit_message_reply_markup(None)

    # When all questions are answered
    if len(state['mcq_user']) == len(state['mcq_correct']):
        score = sum(u == c for u, c in zip(state['mcq_user'], state['mcq_correct']))
        state['tscore'] += score

        await context.bot.send_message(
            chat_id,
            f"✅ You scored {score}/{len(state['mcq_correct'])}. Total score: {state['tscore']}."
        )

        # Reset answer buffers
        state['mcq_user'] = []
        state['mcq_correct'] = []

        # Progress through parts
        if state['stage'] == 'part1':
            state['stage'] = 'part2'
            await generate_passage_and_mcqs(chat_id, context, 'part2', section=state['section'])

        elif state['stage'] == 'part2':
            state['stage'] = 'part3'
            await generate_passage_and_mcqs(chat_id, context, 'part3', section=state['section'])

        elif state['stage'] == 'part3':
            # Move to next section
            next_section = {
                'section1': 'section2',
                'section2': 'section3',
                'section3': 'section4',
            }.get(state['section'])

            if next_section:
                state['section'] = next_section
                state['stage'] = 'part1'
                state['previous_passage'] = ''
                await context.bot.send_message(chat_id, f"🎯 {state['section'].capitalize()} starting...")
                await generate_passage_and_mcqs(chat_id, context, 'part1', section=state['section'])
            else:
                # All sections complete
                await context.bot.send_message(
                    chat_id,
                    f"🎉 All 4 sections completed! Final score: {state['tscore']}"
                )
                state['step'] = 'completed'
# ─── Main Bot Runner ──────────────────────────────────────────────
from telegram.ext import ApplicationBuilder, CommandHandler, CallbackQueryHandler

async def main():
    app = ApplicationBuilder().token(BOT_TOKEN).build()

    app.add_handler(CommandHandler('start', start))
    app.add_handler(CallbackQueryHandler(handle_button))

    print("Bot is polling…")

    await app.initialize()
    await app.start()
    await app.updater.start_polling()

    # Keep running (like idle) using a loop that doesn’t block the current event loop
    while True:
        await asyncio.sleep(1)

# Call the async main if not already running in a blocking loop
await main()

Exception ignored in: <coroutine object Application._bootstrap_initialize at 0x7fee2fa653f0>
Traceback (most recent call last):
  File "<string>", line 1, in <lambda>
KeyError: '__import__'
Exception ignored in: <coroutine object Application._bootstrap_initialize at 0x7fee2fa653f0>
Traceback (most recent call last):
  File "<string>", line 1, in <lambda>
KeyError: '__import__'
Exception ignored in: <coroutine object Application.shutdown at 0x7fee2fa6b030>
Traceback (most recent call last):
  File "<string>", line 1, in <lambda>
KeyError: '__import__'
Exception ignored in: <coroutine object Application.shutdown at 0x7fee2fa6b030>
Traceback (most recent call last):
  File "<string>", line 1, in <lambda>
KeyError: '__import__'
Exception ignored in: <coroutine object Application._bootstrap_initialize at 0x7fee2fa66e30>
Traceback (most recent call last):
  File "<string>", line 1, in <lambda>
KeyError: '__import__'
Exception ignored in: <coroutine object Application.shutdown at 0x7fee2fb2ce

Bot is polling…


ERROR:telegram.ext.Updater:Exception happened while polling for updates.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/telegram/ext/_utils/networkloop.py", line 115, in network_retry_loop
    if not await do_action():
           ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/telegram/ext/_utils/networkloop.py", line 108, in do_action
    return action_cb_task.result()
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/telegram/ext/_updater.py", line 335, in polling_action_cb
    updates = await self.bot.get_updates(
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/telegram/ext/_extbot.py", line 669, in get_updates
    updates = await super().get_updates(
              ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/telegram/_bot.py", line 4601, in get_updates
    await self._post(
  File "/usr/local/lib/python3.11/dist-packages/tele

🎯 Selected topic: inquiring about an apartment for rent


<ipython-input-11-285970a142a9>:318: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  dialogue_json = json.dumps(structured_convo.dict(), indent=2, ensure_ascii=False)
ERROR:telegram.ext.Updater:Exception happened while polling for updates.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/telegram/ext/_utils/networkloop.py", line 115, in network_retry_loop
    if not await do_action():
           ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/telegram/ext/_utils/networkloop.py", line 108, in do_action
    return action_cb_task.result()
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/telegram/ext/_updater.py", line 335, in polling_action_cb
    updates = await self.bot.get_updates(
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/us

...

ERROR:telegram.ext.Updater:Exception happened while polling for updates.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/telegram/ext/_utils/networkloop.py", line 115, in network_retry_loop
    if not await do_action():
           ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/telegram/ext/_utils/networkloop.py", line 108, in do_action
    return action_cb_task.result()
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/telegram/ext/_updater.py", line 335, in polling_action_cb
    updates = await self.bot.get_updates(
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/telegram/ext/_extbot.py", line 669, in get_updates
    updates = await super().get_updates(
              ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/telegram/_bot.py", line 4601, in get_updates
    await self._post(
  File "/usr/local/lib/python3.11/dist-packages/tele

LLM raw output:
 Q1: What time did Jack suggest for viewing the apartment?
a) 9 am
b) 10 am
c) 11 am
d) 12 pm
Answer: b

Q2: What is the condition of the apartment regarding furniture?
a) Fully furnished
b) Partially furnished
c) Unfurnished
d) Furnished upon request
Answer: c

Q3: What is required from Emily if she decides to apply for the apartment after the viewing?
a) Only a security deposit
b) Only an application form
c) An application form, references, and a security deposit
d) An application form and references
Answer: c

Q4: Are pets allowed in the building where the apartment is located?
a) No, pets are not allowed
b) Yes, pets are allowed without any additional fees
c) Yes, pets are allowed with a small additional monthly fee
d) Only certain types of pets are allowed
Answer: c


ERROR:telegram.ext.Updater:Exception happened while polling for updates.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/telegram/ext/_utils/networkloop.py", line 115, in network_retry_loop
    if not await do_action():
           ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/telegram/ext/_utils/networkloop.py", line 108, in do_action
    return action_cb_task.result()
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/telegram/ext/_updater.py", line 335, in polling_action_cb
    updates = await self.bot.get_updates(
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/telegram/ext/_extbot.py", line 669, in get_updates
    updates = await super().get_updates(
              ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/telegram/_bot.py", line 4601, in get_updates
    await self._post(
  File "/usr/local/lib/python3.11/dist-packages/tele

LLM raw output:
 Q1: What is Emily looking forward to doing on Saturday?
a) Meeting Jack for the first time
b) Viewing the apartment
c) Signing a lease
d) Moving into the apartment
Answer: b

Q2: According to Jack, what is the average cost of utilities for the apartment per month?
a) $50
b) $100
c) $200
d) $500
Answer: b

Q3: What amenities are available in the building, according to Jack?
a) Gym and pool
b) Gym and laundry facilities
c) Rooftop garden and parking
d) Cafe and restaurant
Answer: b

Q4: Is the apartment available for a long-term lease, according to Jack?
a) No, only short-term leases are available
b) Yes, but only for a maximum of one year
c) Yes, they are open to negotiating a long-term lease
d) Only for tenants who have rented before
Answer: c


Streaming output truncated to the last 5000 lines.
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/telegram/request/_baserequest.py", line 355, in _request_wrapper
    raise Conflict(message)
telegram.error.Conflict: Conflict: terminated by other getUpdates request; make sure that only one bot instance is running
ERROR:telegram.ext.Updater:Exception happened while polling for updates.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/telegram/ext/_utils/networkloop.py", line 115, in network_retry_loop
    if not await do_action():
           ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/telegram/ext/_utils/networkloop.py", line 108, in do_action
    return action_cb_task.result()
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/telegram/ext/_updater.py", line 335, in polling_action_cb
    updates = await self.bot.get_updates(
              ^^^^^^^^^^^^^^^^^^^